In [19]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import math
import pandas_datareader as pdr
import datetime
pd.core.common.is_list_like = pd.api.types.is_list_like
import yfinance as yf
yf.pdr_override()

#***** pip install PyPortfolioOpt ******
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
#from pypfopt.hierarchical_risk_parity import hrp_portfolio
from pypfopt.value_at_risk import CVAROpt
#from pypfopt import discrete_allocation

In [20]:
# Reading in the data; 
fileName = 'test.csv'
start = datetime.date(2000, 1, 1)#resolve if only input year later
end = datetime.date(2019, 2, 1)
if fileName[fileName.rfind(".")+1:] == 'csv':
     df = pd.read_csv(fileName)
elif fileName[fileName.rfind(".")+1:] == 'xlsx':
     df = pd.read_excel(fileName)
else : 
    print('Wrong File Type')

df.dropna(axis=0,how='any') #drop all rows that have any NaN values
Names = df.iloc[:,0] #first column

tickerList = Names.array #convert Series to array

#retrieve data from yahoo finance
all_data = {}
for ticker in tickerList:
    all_data[ticker] = pdr.get_data_yahoo(ticker, start, end)

#store data in DataFrame
price = DataFrame({tic: data['Adj Close']
                    for tic, data in all_data.items()})
#drop all rows that have any NaN values
price.dropna(axis=0,how='any',inplace=True) 
print(price)

                   SPY        AAPL         GOOG          FB         AMZN  \
Date                                                                       
2012-05-18  112.127739   66.375458   299.078979   38.230000   213.850006   
2012-05-21  114.055038   70.242508   305.908386   34.029999   218.110001   
2012-05-22  114.253792   69.703125   299.278229   31.000000   215.330002   
2012-05-23  114.314316   71.403862   303.592072   32.000000   217.279999   
2012-05-24  114.539001   70.748108   300.702881   33.029999   215.240005   
2012-05-25  114.167389   70.368912   294.660553   31.910000   212.889999   
2012-05-29  115.550171   71.617897   296.060303   28.840000   214.750000   
2012-05-30  113.873535   72.481392   293.016693   28.190001   209.229996   
2012-05-31  113.622864   72.301178   289.345459   29.600000   212.910004   
2012-06-01  110.762215   70.206215   284.423920   27.719999   208.220001   
2012-06-04  110.710373   70.619179   288.214691   26.900000   214.570007   
2012-06-05  

In [21]:
#preparing expected returns and a risk model
returns = price.pct_change().dropna(how="all")
mu = expected_returns.mean_historical_return(price)

S = risk_models.sample_cov(price)
ef = EfficientFrontier(mu, S, weight_bounds=(-1, 1))

In [22]:
#Maximum Sharpe portfolio
weights = ef.max_sharpe()
print(weights)
ef.portfolio_performance(verbose=True)

{'SPY': -0.34661842010850247, 'AAPL': 0.03808649710878209, 'GOOG': 0.22314800276971436, 'FB': 0.1651844664323811, 'AMZN': 0.5322760261376815, 'GE': -0.814185442605675, 'AMD': 0.11566199000731402, 'WMT': 0.32478458170080704, 'BAC': 0.19145848722041015, 'GM': 0.0725040633648291, 'T': -0.18870289522164743, 'UAA': -0.06310865655948833, 'JPM': 0.7495112997533938}
Expected annual return: 54.7%
Annual volatility: 33.4%
Sharpe Ratio: 1.58


(0.5473801475906985, 0.3341906325799393, 1.5780817778144867)

In [23]:
# Long-only minimum volatility portfolio, with a weight cap and regularisation
ef = EfficientFrontier(mu, S, weight_bounds=(0, 0.10), gamma=1)
weights = ef.min_volatility()
print(weights)
ef.portfolio_performance(verbose=True)

{'SPY': 0.08439289136865531, 'AAPL': 0.07935348016251341, 'GOOG': 0.07833333035775442, 'FB': 0.07064585254556766, 'AMZN': 0.07239251678677464, 'GE': 0.08175600549743314, 'AMD': 0.05390004139487133, 'WMT': 0.09060089845149204, 'BAC': 0.0741098061356633, 'GM': 0.07783753065154009, 'T': 0.08996199782572831, 'UAA': 0.06765787468678855, 'JPM': 0.07905777413521796}
Expected annual return: 18.1%
Annual volatility: 15.7%
Sharpe Ratio: 1.03


(0.18131739656841867, 0.1569157269537036, 1.028051169249681)

In [24]:
#Minimize Conditional Value-At-Risk
vr = CVAROpt(returns)
vr.min_cvar()
print(vr.clean_weights())

{'SPY': 0.0, 'AAPL': 0.06957, 'GOOG': 0.23834, 'FB': 0.0, 'AMZN': 0.0, 'GE': 0.1517, 'AMD': 0.0, 'WMT': 0.08196, 'BAC': 0.13274, 'GM': 0.0, 'T': 0.27557, 'UAA': 0.02506, 'JPM': 0.02506}


In [25]:
#Minimize Volatility for a target return of 20%
# A market-neutral Markowitz portfolio
ef = EfficientFrontier(mu, S, weight_bounds=(-1, 1))
weights = ef.efficient_return(target_return=0.20, market_neutral=True)
print(weights)
ef.portfolio_performance(verbose=True)

{'SPY': -0.575633030333052, 'AAPL': 0.023190668768623378, 'GOOG': 0.1083114143289891, 'FB': 0.06592036406281732, 'AMZN': 0.25616958225087433, 'GE': -0.351312629608558, 'AMD': 0.059749096756370315, 'WMT': 0.07471225443050727, 'BAC': 0.11333842666421368, 'GM': 0.03959833943471281, 'T': -0.1500942291265882, 'UAA': -0.019244113266537542, 'JPM': 0.35529385563762766}
Expected annual return: 20.0%
Annual volatility: 13.5%
Sharpe Ratio: 1.33


(0.20000000000020185, 0.13507634964226645, 1.3325796890196568)

In [27]:
# Maximising Return for a target volatility of 20%,
ef = EfficientFrontier(mu, S, weight_bounds=(-1, 1))
weights = ef.efficient_risk(target_risk=0.20)
print(weights)
ef.portfolio_performance(verbose=True)

{'SPY': 0.29602464392643546, 'AAPL': 0.007453338681158058, 'GOOG': 0.09908001231941056, 'FB': 0.09321255646072896, 'AMZN': 0.25014834366055805, 'GE': -0.42340664302767567, 'AMD': 0.04848604936983826, 'WMT': 0.2415102426000048, 'BAC': 0.06389958710804296, 'GM': 0.028078482494197265, 'T': -0.018716909544270542, 'UAA': -0.04205112711355227, 'JPM': 0.3562814230651242}
Expected annual return: 32.5%
Annual volatility: 20.0%
Sharpe Ratio: 1.52


(0.3247788900889803, 0.2000001212892666, 1.5238935262852604)